In [48]:
import pymongo
from datetime import datetime

In [49]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['new_db']
raw_collection = db['raw_news']

In [50]:
raw_data = list(raw_collection.find())

In [51]:
len(raw_data)

228

In [52]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# import spacy


In [53]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AjayPatil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\AjayPatil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AjayPatil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AjayPatil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [54]:
lemmatizer = WordNetLemmatizer()

In [55]:
def preprocess_news(news):
    text = f"{news['title']} {news}['desc] {' '.join(news['content'])}"
    
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\s+"," ", text).strip()
    
    text = text.lower()
    
    tokens = word_tokenize(text)
    
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    lemmatized_token = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(lemmatized_token)

In [56]:
def preprocess_date(date):
    date_obj = datetime.strptime(date, "%B %d, %Y") # strptime str -> datetime obj
    formatted_date = date_obj.strftime("%Y-%m-%d")  # strftime  datetime obj -> formatted str
    return formatted_date

In [57]:
def preprocess_datetime(raw_datetime):
    raw_datetime_split = raw_datetime.split("/")
    date_obj = datetime.strptime(raw_datetime_split[0].strip(), "%B %d, %Y")
    formatted_date = date_obj.strftime("%Y-%m-%d")
    time_obj =  raw_datetime_split[1].strip().replace("IST","")
    formatted_datetime = formatted_date + " " + time_obj
    return formatted_datetime.strip()

# print(preprocess_datetime("January 02, 2025 / 17:53 IST"))

In [58]:
# from yahooquery import search

# def extract_ticker_name(company_name):
#     company_name = company_name.strip()
#     ticker_name = None     
   
#     if company_name:
#         results = search(company_name)        
#         if results and 'quotes' in results:
#             for quote in results['quotes']:
#                 if 'symbol' in quote and 'longname' in quote:
#                     if company_name.lower() in quote['longname'].lower():
#                         ticker_name = quote['symbol']
#                         break      
#     return ticker_name

In [59]:
from yahooquery import search
import time
import requests

def extract_ticker_name(company_name):
    company_name = company_name.strip()
    ticker_name = None  # Initialize as None
    
    # Only proceed if the company name is non-empty
    if company_name:
        retries = 3  # Number of retries in case of failure
        for i in range(retries):
            try:
                # Perform the search
                results = search(company_name)
                
                # If results are found, process them
                if results and 'quotes' in results:
                    for quote in results['quotes']:
                        if 'symbol' in quote and 'longname' in quote:
                            if company_name.lower() in quote['longname'].lower():
                                ticker_name = quote['symbol']
                                break  # Stop once we find a match
                if ticker_name:  # If we found the ticker, break out of the retry loop
                    break
            except (requests.exceptions.RequestException, requests.exceptions.JSONDecodeError) as e:
                # Catch network-related or JSON errors and retry up to `retries` times
                print(f"Attempt {i + 1}/{retries} failed: {e}")
                if i < retries - 1:
                    print("Retrying...")
                    time.sleep(2)  # Wait for 2 seconds before retrying
                else:
                    print("Max retries reached. Returning None.")
                    ticker_name = None  # Set ticker_name to None if the retry limit is reached
                    break  # Exit the loop as we reached the max retries
            except Exception as e:
                # Catch any unexpected exceptions and set ticker_name to None
                print(f"Unexpected error occurred: {e}")
                ticker_name = None
                break  # Exit the loop if any other error occurs
    
    return ticker_name

In [60]:
processed_data = []

In [61]:
for news in raw_data:
    processed_text = preprocess_news(news)
    formatted_date = preprocess_date(news['date'])
    formatted_datetime = preprocess_datetime(news['datetime'])
    if "stock_name" in news:
        ticker_name = extract_ticker_name(news["stock_name"]) 
    
        processed_data.append({
            "_id": news["_id"],
            "processed_text": processed_text,
            "date": formatted_date,
            "datetime": formatted_datetime,
            "stock_name": news.get("stock_name"),
            "ticker_name":ticker_name,        
        })
    else:
        processed_data.append({
                "_id": news["_id"],
                "processed_text": processed_text,
                "date": formatted_date,
                "datetime": formatted_datetime,
                "stock_name": news.get("stock_name"),      
                "ticker_name": None,      
            })

Attempt 1/3 failed: Expecting value: line 1 column 1 (char 0)
Retrying...
Attempt 2/3 failed: Expecting value: line 1 column 1 (char 0)
Retrying...
Attempt 3/3 failed: Expecting value: line 1 column 1 (char 0)
Max retries reached. Returning None.
Attempt 1/3 failed: Expecting value: line 1 column 1 (char 0)
Retrying...
Attempt 2/3 failed: Expecting value: line 1 column 1 (char 0)
Retrying...
Attempt 3/3 failed: Expecting value: line 1 column 1 (char 0)
Max retries reached. Returning None.
Attempt 1/3 failed: Expecting value: line 1 column 1 (char 0)
Retrying...
Attempt 2/3 failed: Expecting value: line 1 column 1 (char 0)
Retrying...
Attempt 3/3 failed: Expecting value: line 1 column 1 (char 0)
Max retries reached. Returning None.
Attempt 1/3 failed: Expecting value: line 1 column 1 (char 0)
Retrying...
Attempt 2/3 failed: Expecting value: line 1 column 1 (char 0)
Retrying...
Attempt 3/3 failed: Expecting value: line 1 column 1 (char 0)
Max retries reached. Returning None.
Attempt 1/3 

In [62]:
processed_data[4]

{'_id': ObjectId('6777a0d78581574bb8d03acd'),
 'processed_text': 'shiva texyarn share zoom firm secures major order indian air force id objectidadbbdacd title shiva texyarn share zoom firm secures major order indian air force desc indian air force assigned order valued r crore shiva texyarn supply pair nbc date january datetime january ist content shiva texyarn ltdxastock rose percent january xafollowing significant order win indian air force total order value order r crore inclusive duty tax per exchange filing contract dept military affair supply pair nbc nuclear biological chemical suit permeable mk v delivery specialised suit scheduled completed january august per statement company order testament shiva texyarns expertise manufacturing protective gear ability meet stringent requirement indian armed force company confirmed order fall related party transaction qfy company registered net sale r crore percent r crore september meanwhile quarterly net profit stood r crore percent corres

In [63]:
collection = db['processed_news']
x = collection.insert_many(processed_data)

In [64]:
# x.inserted_ids

In [65]:
client.close()